### Building evasion vs. mean time interval : by Bus by Service - THIS IS A TEST

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from Utils import TransantiagoConstants

In [3]:
first_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/1st_quarter.xlsx"
second_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/2nd_quarter.xlsx"
third_quarter_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/06_RFA/01_EvasionTrimestral/01_analisis/3rd_quarter.xlsx"
codes_path = "C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/04_DTPM/codes_services.xlsx"

In [4]:
first_quarter_evasion = pd.read_excel(first_quarter_path, encoding = 'latin-1')
second_quarter_evasion = pd.read_excel(second_quarter_path, encoding = 'latin-1')
third_quarter_evasion = pd.read_excel(third_quarter_path, encoding = 'latin-1')
codes = pd.read_excel(codes_path, encoding = 'latin-1')

In [5]:
first_quarter_evasion['TIEMPO'] = first_quarter_evasion['HORA'].astype(str)+':'+first_quarter_evasion['MINUTOS'].astype(str)+':00'
second_quarter_evasion['TIEMPO'] = second_quarter_evasion['HORA'].astype(str)+':'+second_quarter_evasion['MINUTOS'].astype(str)+':00'
third_quarter_evasion['TIEMPO'] = third_quarter_evasion['HORA'].astype(str)+':'+third_quarter_evasion['MINUTOS'].astype(str)+':00'

In [6]:
first_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
second_quarter_evasion.columns = ['FECHA', 'SERVICIO','TIPO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
third_quarter_evasion.columns = ['FECHA', 'SERVICIO','PATENTE','PUERTAS','N_PUERTA','LUGAR_INICIO','HORA_INICIO','HORA','MINUTO','INGRESAN','NO_VALIDAN','TP','TIEMPO']
del second_quarter_evasion['TIPO']

In [7]:
type(first_quarter_evasion.loc[0,'TIEMPO'])

str

In [8]:
type(first_quarter_evasion.loc[0,'FECHA'])

pandas._libs.tslib.Timestamp

In [9]:
frames = [first_quarter_evasion, second_quarter_evasion, third_quarter_evasion]
evasion = pd.concat(frames, keys=['first', 'second', 'third'])

In [10]:
codes = codes.rename(columns = {'USER_CODE':'SERVICIO'})
del codes['DIRECTION']

In [11]:
evasion = pd.merge(evasion,codes, on=['SERVICIO'], how='left')

In [12]:
evasion.loc[evasion['TS_CODE'].isnull(),'SERVICIO'].unique()

array(['D06'], dtype=object)

* THIS IS HARDCODED

In [13]:
evasion.loc[evasion['TS_CODE'].isnull(),'TS_CODE']='446'

In [ ]:
evasion.head()

* Processing evasion-ddbb before merging

In [15]:
evasion['PATENTE'] =  evasion['PATENTE'].str.replace(' ','')

evasion['SERVICIO_TMP'] = evasion['SERVICIO'].apply(str)
evasion['TS_CODE_TMP'] = evasion['TS_CODE'].apply(str)

del evasion['SERVICIO']
del evasion['TS_CODE']

evasion = evasion.rename(columns = {'SERVICIO_TMP':'SERVICIO_USUARIO', 'TS_CODE_TMP':'SERVICIO'})
evasion_paradero = evasion[evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]

In [16]:
type(evasion['SERVICIO'][0])

str

* Getting evasion by bus by service

In [17]:
summary = evasion_paradero_first.groupby(['PATENTE','SERVICIO','FECHA'])['INGRESAN','NO_VALIDAN'].agg(['count','sum'])
summary.columns=['EVASION_COUNT', 'TOTAL_INGRESAN', 'tmp_count' ,'TOTAL_NO_VALIDAN']
del summary['tmp_count']
summary.reset_index(inplace=True)

* Building complete etapas ddbb.

In [18]:
common_dates = TransantiagoConstants.common_dates

In [19]:
reduced_etapas = pd.DataFrame()

for date in common_dates:
    base_input_path = 'C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/01_SSH/02_TIMESBYBUSBYSERVICE/01_ORIGINAL/'
    file_name = date + '-meanByBusByService.csv'
    grouped_by_bus = pd.read_csv(base_input_path+file_name, encoding = 'latin-1',sep =";")
    reduced_etapas = pd.concat([reduced_etapas,grouped_by_bus])


* Processing etapas ddbb before merging

In [20]:
del reduced_etapas['Unnamed: 0']
reduced_etapas = reduced_etapas.rename(columns = {'fecha':'FECHA', 'sitio_subida':'PATENTE', 'count': 'ETAPAS_COUNT', 'servicio_subida' : 'SERVICIO'})
reduced_etapas['FECHA'] = pd.to_datetime(reduced_etapas.FECHA)
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace("-", "")
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace(" ", "")

In [21]:
reduced_etapas.reset_index(drop=True,inplace=True)

In [22]:
type(reduced_etapas.loc[0,'FECHA'])

pandas._libs.tslib.Timestamp

## <strong>TEST MERGING</strong>

1 - BASICS 

In [23]:
evasion_vs_mean = pd.merge(summary,reduced_etapas, on=['PATENTE','SERVICIO','FECHA'], how='left')

In [24]:
evasion_vs_mean_length = len(evasion_vs_mean.index)
print('The total number of rows in evasion vs. mean ddbb is: ' +str(evasion_vs_mean_length))

The total number of rows in evasion vs. mean ddbb is: 1213


* Getting rid of missing values

In [25]:
clean_evasion_vs_mean = evasion_vs_mean[evasion_vs_mean['mean']>0]

In [26]:
clean_evasion_vs_mean_length = len(clean_evasion_vs_mean.index)
print('The total number of rows matching in evasion vs. mean ddbb is: ' + str(clean_evasion_vs_mean_length))
print('The percentage of matching is: ' + str(clean_evasion_vs_mean_length/evasion_vs_mean_length*100) + '%')

The total number of rows matching in evasion vs. mean ddbb is: 482
The percentage of matching is: 39.73619126133553%


= END BASICS =

2 - NUMBER OF OBSERVATIONS MISMATCHING

In [27]:
total_mismatching = evasion_vs_mean_length - clean_evasion_vs_mean_length
print('The total number of rows mismatching in evasion vs. mean ddbb is: ' + str(total_mismatching))

The total number of rows mismatching in evasion vs. mean ddbb is: 731


= END NUMBER OF OBSERVATIONS MISMATCHING =

3 - CHECKING CONDITIONS

In [28]:
within_dates_evasion_vs_mean = evasion_vs_mean[(evasion_vs_mean['FECHA']>=pd.to_datetime('2017-03-09'))&(evasion_vs_mean['FECHA']<=pd.to_datetime('2017-08-24'))]

In [29]:
within_dates_evasion_vs_mean_length = len(within_dates_evasion_vs_mean.index)
print('The number of rows within common dates range is: ' + str(within_dates_evasion_vs_mean_length))

The number of rows within common dates range is: 799


In [30]:
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]

In [31]:
common_dates_evasion_vs_mean = within_dates_evasion_vs_mean[within_dates_evasion_vs_mean['FECHA'].isin(common_dates_timestamp)]

In [32]:
common_dates_evasion_vs_mean_length = len(common_dates_evasion_vs_mean.index)
print('The total number of rows within common dates range and in common dates is: ' + str(common_dates_evasion_vs_mean_length))

The total number of rows within common dates range and in common dates is: 517


In [33]:
print('IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS ' + str(evasion_vs_mean_length) + '. FROM THIS, ' + 
     str(within_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, ' + 
     str(common_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE ' +
     str(common_dates_evasion_vs_mean_length-clean_evasion_vs_mean_length) + ' ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES') 

IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS 1213. FROM THIS, 799 ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, 517 ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE 35 ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES


= END CHECKING CONDITIONS =

4 - CHECKING PROBLEMATIC ROWS AND LOOKING INTO REDUCED_ETAPAS SIMPLIFIED_ETAPAS FILES

In [ ]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['sum'].notnull()].head(4)

In [ ]:
common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['sum'].isnull()].head(4)

* Checking in reduced_etapas ...
* Checking for PATENTE == BBKB-12, SERVICIO == F11, FECHA == 2017-06-14	> Doesn't exist
* Checking for PATENTE == BFHB-25, SERVICIO == 212, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BGTL-44, SERVICIO == 334, FECHA == 2017-08-17 > Doesn't exist
* Checking for PATENTE == BGTL-77, SERVICIO == 339, FECHA == 2017-06-14 > Doesn't exist

* <font color='green'><strong>Merge is correct.</strong></font> These 35 rows should be checked in simplified_etapas files

In [36]:
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [37]:
date = '2017-06-14'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
filtered_df = etapas_builder.runLightCompleteProcess()

Not found in turnstile database: 517700


In [38]:
filtered_df[filtered_df['sitio_subida']=='BBKB-12']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete
153,405460,2036648403,3,BUS,2017-06-14 22:27:10,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:02,2.0,False,False
154,293942,930125355,2,BUS,2017-06-14 22:27:13,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
155,465616,2483164525,2,BUS,2017-06-14 22:27:17,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:04,4.0,False,False
156,78158,52372738,3,BUS,2017-06-14 22:27:20,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
157,193185,424007954,3,BUS,2017-06-14 22:27:23,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
158,23227,31833762,3,BUS,2017-06-14 22:27:26,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:03,3.0,False,False
159,224809,473063026,3,BUS,2017-06-14 22:27:28,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:02,2.0,False,False
160,359613,1524637928,2,BUS,2017-06-14 22:27:28,F29 00R,T-34-270-SN-10,BBKB-12,NaT,True,True,True,00:00:00,0.0,False,False


In [39]:
filtered_df[filtered_df['sitio_subida']=='BFHB-25']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


In [40]:
filtered_df[filtered_df['sitio_subida']=='BGTL-77']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


In [41]:
filtered_df[filtered_df['sitio_subida']=='BBKB-13']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete
163,63592,47993730,2,BUS,2017-06-14 21:43:31,F11 00R,E-34-270-NS-70,BBKB-13,NaT,True,True,True,00:00:22,22.0,False,False


* Checking in simplified_etapas ...
* Checking for PATENTE == BBKB-12, SERVICIO == F11, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BFHB-25, SERVICIO == 212, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BGTL-77, SERVICIO == 339, FECHA == 2017-06-14 > Doesn't exist
* Checking for PATENTE == BBKB-13, SERVICIO == F11, FECHA == 2017-06-14 > It exists
* <font color='green'><strong>Merge is correct. TEST PASSED</strong></font>

In [42]:
date = '2017-08-17'
etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
filtered_df = etapas_builder.runLightCompleteProcess()

Not found in turnstile database: 1873694


In [43]:
filtered_df[filtered_df['sitio_subida']=='BGTL-44']

,index,id,nviaje,tipo_transporte,t_subida,servicio_subida,par_subida,sitio_subida,fecha_instalacion,mismo_paradero,misma_patente,mismo_servicio,diferencia_tiempo,diferencia_tiempo_secs,si_torniquete,si_2017_torniquete


* Checking in simplified_etapas ...
* Checking for PATENTE == BGTL-44, SERVICIO == 334, FECHA == 2017-08-17 > Doesn't exist
* <font color='green'><strong>Merge is correct. TEST PASSED</strong></font>

### Closed